# Calculate the stationary internal tide component

# Dask worker setup instructions

https://github.com/mrayson/pawsey-containers/tree/master/hpc-python/jupyter-sfoda

These settings do the job:

 1. `dask-scheduler --scheduler-file scheduler-$HOSTNAME.json --idle-timeout 0`
 2. `ssh localhost "cd /software/projects/pawsey0106/mrayson/projects && sbatch --ntasks=16 -c 32 start_worker.slm scheduler-$HOSTNAME.json ./"`


In [1]:
hostname = !echo $(hostname)
hostname[0]

'nid002537'

In [2]:
import os
from distributed import Client
client=Client(scheduler_file='../../scheduler-' + hostname[0] + '.json')
client

<Client: 'tcp://10.253.136.193:8786' processes=16 threads=512, memory=859.83 GiB>

In [3]:
# client.restart() # This clears all data

In [4]:
import sfoda.suntans.sunxarray as sxr

import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from cmocean import cm
from s3fs import S3FileSystem, S3Map

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-eolmkp86 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
s3login = '/home/mrayson/code/s3.acacia.login'

with open(s3login) as f:
    lines = f.readlines()

url, key, secret = [ff.strip('\n') for ff in lines[0:3]]
s3 = S3FileSystem(client_kwargs={'endpoint_url':url},
            key=key,
            secret=secret)


In [6]:
####
# f = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_ssh_steric_filtered.zarr/", s3=s3) # 6 hour cutoff
# f = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_ssh_steric_filtered_v2.zarr/", s3=s3) # 4 hour cutoff
# varname = 'eta_steric_bandpass'
# zarrfile = 'NWS_2km_GLORYS_2013_2014_ssh_steric_filtered_seasonaltides.zarr'

f = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_surface.zarr/", s3=s3)
# varname = 'vc'
# longname = 'northward velocity'
# varname = 'uc'
# longname = 'eastward velocity'
varname = 'eta'
longname = 'free surface height'
dropvars=['vc','uc','temp','eta']

# f = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_ssh_filtered.zarr/", s3=s3)
# varname = 'eta_bandpass'
# dropvars = ['eta_steric_lowpass','eta_steric_bandpass',]
# longname = 'phase_locked steric sea surface height'
# dropvars = ['eta_lowpass','eta_bandpass',]
# longname = 'phase_locked  sea surface height'

seasonal_freq_list = ['M2','S2','K1','O1']
freq_list = ['N2','Q1']
na = 2 # Number of annual harmonics to include
# zarrfile = 'NWS_2km_GLORYS_2013_2014_ssh_filtered_seasonaltides.zarr'
zarrfile = 'NWS_2km_GLORYS_2013_2014_{}_seasonaltides.zarr'.format(varname)

# seasonal_freq_list = []
# freq_list = ['M2','S2','N2','K2','K1','O1','P1','Q1']
#na = 0 # Number of annual harmonics to include
# zarrfile = 'NWS_2km_GLORYS_2013_2014_ssh_steric_filtered_astrotides.zarr'
# zarrfile = 'NWS_2km_GLORYS_2013_2014_ssh_filtered_astrotides.zarr'
#zarrfile = 'NWS_2km_GLORYS_2013_2014_vc_astrotides.zarr'

mode='w'
####
zarrfile

'NWS_2km_GLORYS_2013_2014_eta_seasonaltides.zarr'

In [7]:
ds = xr.open_dataset(f, engine='zarr', chunks={})

# These need filling in??
ds.suntans.xv=ds.xv.values
ds.suntans.yv=ds.yv.values

# Need to fix the time because the restart at each month lead to non-continuous time stepping
# Find and remove duplicate time entries
tsec = (ds.time.values-ds.time.values[0]).astype(float)/1e9
dt = np.ones(tsec.shape)

dt[1:] = tsec[1:]-tsec[0:-1]
dt[0] = dt[1]*1

#plt.plot(dt)

#badidx = dt<dt.max()
#dt[np.argwhere(badidx)]

remove_idx = dt==60.
# fix_idx = dt==3540.

# myds.time.isel(time=fix_idx) + np.timedelta64(60,'s')
ds2 = ds.isel(time=~remove_idx).chunk({'time':-1})

Changing the _FillValue from -999999 to 999999


In [8]:
# ds['eta_bandpass'].isel(Nc=10000).plot()
# ds['eta_lowpass'].isel(Nc=10000).plot()


In [9]:
# ds['eta_steric_lowpass'].isel(Nc=10000).plot()
# ds['eta_steric_bandpass'].isel(Nc=10000).plot()

In [10]:
# ds['eta_steric_tidal'].isel(Nc=10000).plot()
# (ds['eta_steric_bandpass'].isel(Nc=10000)-ds['eta_steric_tidal'].isel(Nc=10000)).plot()

# ds['eta_tidal'].isel(Nc=10000).plot()
# (ds['eta_bandpass'].isel(Nc=10000)-ds['eta_tidal'].isel(Nc=10000)).plot()

In [11]:
import sfoda.utils.harmonic_analysis as ha
from sfoda.utils.othertime import SecondsSince
from datetime import datetime
import dask


In [12]:
def build_lhs_dask(t,frq):
    """
    Construct matrix A
    """
    nt=t.shape[0]
    frq = np.array(frq)
    
    nf=frq.shape[0]
    nff=nf*2+1
    A=np.ones((nt,nff))
    for ff in range(0,nf):
        A[:,ff*2+1]=np.cos(frq[ff]*t)
        A[:,ff*2+2]=np.sin(frq[ff]*t)

    return dask.array.from_array(A, chunks=(A.shape[0],A.shape[1]))

def lstsq_dask(A,y):    
    """    
    Solve the least square problem

    Return:
        the complex amplitude 
        the mean
    """
    N=A.shape[1]
    b,res,rank,s = dask.array.linalg.lstsq(A,y)

    return b


def harmonic_fit_dask(X, t, omega):
           
    A = build_lhs_dask(t, omega)

    
    #print('A: ',A.shape)
    
    # Remove NaNs
    #X[dask.array.isnan(X)] = 0.

    # Least-squares matrix approach
    b = lstsq_dask(A, X) # This works on all columns of X!!
    #print('b', b.shape)

    return b

twopi = 2*np.pi
tdaysec = 86400.

def get_seasonal_freqs(omega, na, omega_A=twopi/(365*tdaysec)):
    frq_all =[]
    for ff in omega:
        for n in range(-na,na+1):
            frq_all.append(ff+n*omega_A)
            
    return frq_all

def nonstat_harmonic_fit_dask(X, t, frq_all):

    Y = harmonic_fit_dask(X, t, frq_all)
    aa = Y[0,...]
    Aa = Y[1::2,...]
    Ba = Y[2::2,...]
    
    return aa, Aa, Ba



In [13]:
# Do the harmonic fits
reftime = datetime(1990,1,1)

time = SecondsSince(ds2.time.values, basetime=reftime)
sfrq, names = ha.getTideFreq(seasonal_freq_list)
frq_all = get_seasonal_freqs(sfrq, na)
frq, names = ha.getTideFreq(freq_list) # Add on frequencies without seasonal bits (usually Q1 and N2)
for ff in frq:
    frq_all.append(ff)
    
aa, Aa, Ba = nonstat_harmonic_fit_dask(ds2[varname], time, frq_all)
# aa, Aa, Ba, frq_all = nonstat_harmonic_fit_dask(ds['eta_bandpass'], time, frq, na)

/opt/conda/lib/python3.10/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


In [14]:
# aa.rechunk((2000,))

In [15]:
def harmonic_pred_dask(aa, Aa, Ba, omega, tdays):
    nomega = len(omega)
    nt = tdays.shape[0]
    amp = dask.array.ones_like(tdays)[:,None] * aa
    for ii in range(nomega):
        amp += Aa[ii,...]*np.cos(omega[ii]*tdays)[:,None] + Ba[ii,...]*np.sin(omega[ii]*tdays)[:,None]
    
    return amp

In [16]:
eta_steric_tidal = harmonic_pred_dask(aa.rechunk(2000), Aa.rechunk((-1,2000)), Ba.rechunk((-1,2000)), frq_all, time)
eta_steric_tidal

dask.array<add, shape=(8761, 225368), dtype=float64, chunksize=(8761, 2000), chunktype=numpy.ndarray>

In [17]:
# eta_steric_tidal_ = eta_steric_tidal.persist()

In [18]:
# # Create an output data set

dsout = ds2.drop_vars(dropvars)

dsout['{}_phase_locked'.format(varname)] = xr.DataArray(eta_steric_tidal, dims=('time','Nc'), 
                                           attrs=
                                {'long_name':longname,'units':'m','location':'face',
                                'seasonal_frequencies':seasonal_freq_list,
                                'tidal_frequencies':freq_list,
                                'number_of_annual_harmonics':na} )#.chunk({'Nc':2000})

# dsout['eta_steric_non_phase_locked'] =  \
#                                 xr.DataArray(ds['eta_steric_bandpass'] - eta_steric_tidal, dims=('time','Nc'), 
#                                            attrs=
#                                 {'long_name':'non_phase_locked steric sea surface height','units':'m','location':'face',
#                                 'tidal_frequencies':freq_list,
#                                 'number_of_annual_harmonics':na} )

dsout

# Create an output data set
# dsout = ds.drop_vars(['eta_lowpass','eta_bandpass'])

# dsout['eta_tidal'] = xr.DataArray(eta_steric_tidal, dims=('time','Nc'), 
#                                            attrs=
#                                 {'long_name':'steric sea surface height','units':'m','location':'face',
#                                 'tidal frequencies':freq_list} )


# dsout

<xarray.Dataset>
Dimensions:           (Nc: 225368, numsides: 8, Nk: 80, time: 8761, Np: 454016)
Coordinates:
  * time              (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-07-01
    z_r               (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
Dimensions without coordinates: Nc, numsides, Nk, Np
Data variables:
    cells             (Nc, numsides) int32 dask.array<chunksize=(2000, 8), meta=np.ndarray>
    dv                (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    dz                (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
    nfaces            (Nc) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
    xp                (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    xv                (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    yp                (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    yv                (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    eta_phase_locked  (time, Nc) float64 dask.array<chunksize=(8761, 2000), meta=np.ndarray>
Attributes:
    Author:       mrayson
    Description:  SUNTANS surface fields

In [19]:
# Save local file
# outfile = '/scratch/pawsey0106/mrayson/SUNTANS_NWS_2km/'+zarrfile
# # outfile = '/scratch/pawsey0106/mrayson/SUNTANS_NWS_2km/NWS_2km_GLORYS_2013_2014_ssh_filtered.zarr'

# dsout.to_zarr(outfile, mode=mode, append_dim=None, compute=True)
# outfile

In [20]:
# Save time chunked output
# dsout = xr.open_dataset(outfile,engine='zarr') # Load above to avoid recomputing

d = S3Map("uwaoceanprocesses-suntans/"+zarrfile, s3=s3)
# d = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_ssh_filtered.zarr", s3=s3)

dsout.to_zarr(store=d, mode=mode, compute=True, safe_chunks=False)

2023-08-22 16:03:12,221 - distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('store-map-58b504fb6748340d24936c2336cda43d', 0, 82)": ()}


In [21]:
# dropvars=['eta_steric_phase_locked']
# outvar = 'eta_steric_non_phase_locked'
# varbpass = 'eta_steric_bandpass'
dropvars=['{}_phase_locked'.format(varname)]
outvar = '{}_non_phase_locked'.format(varname)
# varbpass = 'eta_bandpass'
varbpass = varname

dsout = dsout.drop_vars(dropvars)
dsout[outvar] =  \
                                xr.DataArray(ds2[varbpass] - eta_steric_tidal, dims=('time','Nc'), 
                                           attrs=
                                {'long_name':'non_phase_locked sea surface height','units':'m','location':'face',
                                 'seasonal_frequencies':seasonal_freq_list,
                                'tidal_frequencies':freq_list,
                                'number_of_annual_harmonics':na} )

dsout.to_zarr(store=d, mode='a', compute=True, safe_chunks=False)
dsout

<xarray.Dataset>
Dimensions:               (Nc: 225368, numsides: 8, Nk: 80, time: 8761,
                           Np: 454016)
Coordinates:
  * time                  (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-...
    z_r                   (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
Dimensions without coordinates: Nc, numsides, Nk, Np
Data variables:
    cells                 (Nc, numsides) int32 dask.array<chunksize=(2000, 8), meta=np.ndarray>
    dv                    (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    dz                    (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
    nfaces                (Nc) int32 dask.array<chunksize=(2000,), meta=np.ndarray>
    xp                    (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    xv                    (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    yp                    (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    yv                    (Nc) float64 dask.array<chunksize=(2000,), meta=np.ndarray>
    eta_non_phase_locked  (time, Nc) float64 dask.array<chunksize=(8761, 2000), meta=np.ndarray>
Attributes:
    Author:       mrayson
    Description:  SUNTANS surface fields

In [22]:
dstest =xr.open_dataset(d, engine='zarr')
dstest

<xarray.Dataset>
Dimensions:               (Nc: 225368, numsides: 8, Nk: 80, time: 8761,
                           Np: 454016)
Coordinates:
  * time                  (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-...
    z_r                   (Nk) float64 ...
Dimensions without coordinates: Nc, numsides, Nk, Np
Data variables:
    cells                 (Nc, numsides) int32 ...
    dv                    (Nc) float64 ...
    dz                    (Nk) float64 ...
    eta_non_phase_locked  (time, Nc) float64 ...
    eta_phase_locked      (time, Nc) float64 ...
    nfaces                (Nc) int32 ...
    xp                    (Np) float64 ...
    xv                    (Nc) float64 ...
    yp                    (Np) float64 ...
    yv                    (Nc) float64 ...
Attributes:
    Author:       mrayson
    Description:  SUNTANS surface fields

# Chunk along space and re-save

Mainly used for spatial plotting

In [38]:
from sfoda.utils.myproj import MyProj
# Convert the coordinates
P=MyProj('merc')


In [39]:
ds_c = ds
ds_c['cells']= xr.where(ds.cells==999999, -999999, ds.suntans.cells)

ds_c['xv'][:], ds_c['yv'][:] = P.to_xy(ds['xv'], ds['yv'])
ds_c['xp'][:], ds_c['yp'][:] = P.to_xy(ds['xp'], ds['yp'])

ds_c = ds_c.chunk({'Nc':-1,'time':96})
ds_c#, ds_c['cells'].values


<xarray.Dataset>
Dimensions:       (Nc: 225368, numsides: 8, Nk: 80, time: 8761, Np: 454016)
Coordinates:
  * time          (time) datetime64[ns] 2013-07-01T00:01:00 ... 2014-07-01
    z_r           (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
Dimensions without coordinates: Nc, numsides, Nk, Np
Data variables:
    cells         (Nc, numsides) int32 dask.array<chunksize=(225368, 8), meta=np.ndarray>
    dv            (Nc) float64 dask.array<chunksize=(225368,), meta=np.ndarray>
    dz            (Nk) float64 dask.array<chunksize=(80,), meta=np.ndarray>
    eta           (time, Nc) float64 dask.array<chunksize=(96, 225368), meta=np.ndarray>
    eta_bandpass  (time, Nc) float64 dask.array<chunksize=(96, 225368), meta=np.ndarray>
    eta_lowpass   (time, Nc) float64 dask.array<chunksize=(96, 225368), meta=np.ndarray>
    eta_tidal     (time, Nc) float64 dask.array<chunksize=(96, 225368), meta=np.ndarray>
    nfaces        (Nc) int32 dask.array<chunksize=(225368,), meta=np.ndarray>
    xp            (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    xv            (Nc) float64 dask.array<chunksize=(225368,), meta=np.ndarray>
    yp            (Np) float64 dask.array<chunksize=(454016,), meta=np.ndarray>
    yv            (Nc) float64 dask.array<chunksize=(225368,), meta=np.ndarray>
Attributes:
    Author:       mrayson
    Description:  SUNTANS surface fields

In [40]:
# ds_c['eta_steric_bandpass'].encoding['chunks']={'chunks':(96,225368) }
# ds_c['eta_steric_bandpass'].encoding['preferred_chunks']={'time': 96, 'Nc': 225368},
# ds_c['eta_steric_lowpass'].encoding['chunks']={'chunks':(96,225368) }
# ds_c['eta_steric_lowpass'].encoding['preferred_chunks']={'time': 96, 'Nc': 225368},
# ds_c['eta_steric_tidal'].encoding['chunks']={'chunks':(96,225368) }
# ds_c['eta_steric_tidal'].encoding['preferred_chunks']={'time': 96, 'Nc': 225368},
# ds_c['eta_steric_bandpass'].encoding

ds_c['eta_bandpass'].encoding['chunks']={'chunks':(96,225368) }
ds_c['eta_bandpass'].encoding['preferred_chunks']={'time': 96, 'Nc': 225368},
ds_c['eta_lowpass'].encoding['chunks']={'chunks':(96,225368) }
ds_c['eta_lowpass'].encoding['preferred_chunks']={'time': 96, 'Nc': 225368},
ds_c['eta_tidal'].encoding['chunks']={'chunks':(96,225368) }
ds_c['eta_tidal'].encoding['preferred_chunks']={'time': 96, 'Nc': 225368},
ds_c['eta_bandpass'].encoding

{'chunks': {'chunks': (96, 225368)},
 'preferred_chunks': ({'time': 96, 'Nc': 225368},),
 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0),
 'filters': None,
 '_FillValue': nan,
 'dtype': dtype('float64')}

In [41]:
# Save time chunked output
# d = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_ssh_steric_filtered_spacechunk.zarr", s3=s3)
d = S3Map("uwaoceanprocesses-suntans/NWS_2km_GLORYS_2013_2014_ssh_filtered_spacechunk.zarr", s3=s3)

ds_c.to_zarr(store=d, mode='w', compute=True, safe_chunks=False)

In [21]:
client.restart()

<Client: 'tcp://10.253.136.193:8786' processes=14 threads=448, memory=752.35 GiB>